In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import chardet as chardet
import dask as dask
import dask.dataframe as dd

import sys
import subprocess
from pathlib import Path

# Import Dependencies
import psycopg2
from pymongo import MongoClient
from pprint import pprint
from utils import createCSV, createCSVUsingDask

In [8]:
df = pd.read_csv("Resources/us_vehicle.csv" )

C:\Users\darji\AppData\Local\Temp\ipykernel_25068\3950099996.py:1: DtypeWarning: Columns (0,2,3,4,5,7,9,11,13,15,17,19,21,23,25,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Resources/us_vehicle.csv" )


         STATE STATENAME  ST_CASE  VEH_NO  VE_FORMS  NUMOCCS NUMOCCSNAME  \
count   228481    228481   228481  228481    228481   228481      227283   
unique      92        52    71162     190        38       81          55   
top          6     Texas   480549       1         2        1           1   
freq     18141     23530      132   84582     61000    90713      121020   

        HIT_RUN HIT_RUNNAME REG_STAT  ... VSPD_LIM  VSPD_LIMNAME  VALIGN  \
count    228481      228481   228481  ...   228481        228481  228481   
unique        7           4      130  ...       41            21      15   
top           0          No        6  ...       55        55 MPH       1   
freq     124449      217927    16714  ...    32931         55980  101431   

        VALIGNNAME VPROFILE  VPROFILENAME VPAVETYP  DEATHS DR_DRINK  \
count       228481   228481        228481   228481  228481   228481   
unique          10       19            10       19      21        5   
top       Straight       

In [2]:
def createUSAccidentData(base_path):
    createCSV(f"{base_path}/2022/accident.csv","Resources/accident_labels.csv","us_accident","w")
    createCSV(f"{base_path}/2021/accident.csv","Resources/accident_labels.csv","us_accident","a")
    createCSV(f"{base_path}/2020/accident.csv","Resources/accident_labels.csv","us_accident","a")
    createCSV(f"{base_path}/2019/accident.csv","Resources/accident_labels.csv","us_accident","a")

In [3]:
import import_ipynb as ipynb
def loadUSAccidentDataToMongoDB():
    object_names = ['us_accident']
    # Pass the collection names
    from mongodb_load_databases import createMongoDB, validateMongoDB
    db_name = 'us_accidents_db'
    createMongoDB(db_name, object_names)
    validateMongoDB(db_name)

In [4]:
import import_ipynb as ipynb
def loadUSAccidentDataToMongoDBCluster():
    object_names = ['us_accident']
    # Pass the collection names
    from mongodb_load_databases import createMongoDBCluster, validateMongoDBCluster
    db_name = 'us_accidents_db'
    createMongoDBCluster(db_name, object_names)
    validateMongoDBCluster(db_name)

In [5]:
base_path = Path("Resources/accident_data")
createUSAccidentData(base_path)
loadUSAccidentDataToMongoDB()

Starting to read the File
################Labels to Save###################### Index(['STATE', 'STATENAME', 'ST_CASE', 'VE_TOTAL', 'PVH_INVL', 'PEDS',
       'PERSONS', 'PERMVIT', 'PERNOTMVIT', 'COUNTYNAME', 'CITYNAME', 'DAY',
       'MONTH', 'MONTHNAME', 'YEAR', 'DAY_WEEKNAME', 'HOURNAME', 'MINUTE',
       'NHSNAME', 'ROUTENAME', 'TWAY_ID', 'RUR_URBNAME', 'FUNC_SYSNAME',
       'RD_OWNERNAME', 'WEATHERNAME', 'LATITUDE', 'LONGITUD', 'SP_JURNAME',
       'HARM_EVNAME', 'RELJCT1NAME', 'WRK_ZONENAME', 'REL_ROADNAME',
       'LGT_CONDNAME', 'FATALS'],
      dtype='object')
################Updated CSV File#################        STATE STATENAME  ST_CASE  PEDS  PERNOTMVIT  VE_TOTAL  PVH_INVL  \
0          1   Alabama    10001     0           0         2         0   
1          1   Alabama    10002     0           0         2         0   
2          1   Alabama    10003     0           0         1         0   
3          1   Alabama    10004     0           0         1         0   
4        

In [6]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# confirm that the "autosaurus" database is in MongoDB
print(mongo.list_database_names())

['admin', 'config', 'local', 'us_accidents_db']


In [7]:
# assign the database to a variable name
db = mongo['us_accidents_db']
# review the collections in our new database
print(db.list_collection_names())

['us_accident']


In [8]:
# assign each collection to a variable
accident = db['us_accident']

In [9]:
#Calculate total fatalities over all years
total_fatalities = accident.count_documents({})
#Display the data
print("Total Fatalities: ", total_fatalities)

Total Fatalities:  148428


In [10]:
#Calculate fatalities by year
pipeline_year = [
    {"$group": {"_id": "$YEAR", "total_fatalities": {"$sum": 1}}},
    {"$sort": {"_id": -1}}  # Sort by year descending
]
fatalities_by_year = list(accident.aggregate(pipeline_year))
df_year = pd.DataFrame(fatalities_by_year).rename(columns={'_id': 'Year'})

print("\nFatalities by Year:")
print(df_year)


Fatalities by Year:
   Year  total_fatalities
0  2022             39221
1  2021             39785
2  2020             35935
3  2019             33487


In [11]:
pipeline_state = [
    {"$group":{"_id": "$STATENAME", "total_fatalities": {"$sum": 1}}},
    {"$sort": {"total_fatalities": -1}}  # Sort by fatalities descending
]
fatalities_by_state = list(accident.aggregate(pipeline_state))
df_state = pd.DataFrame(fatalities_by_state).rename(columns={'_id': 'State'})
print("\nFatalities by State:")
print(df_state)



Fatalities by State:
                   State  total_fatalities
0             California             15405
1                  Texas             14854
2                Florida             12800
3                Georgia              6254
4         North Carolina              5836
5                   Ohio              4615
6              Tennessee              4604
7               Illinois              4381
8           Pennsylvania              4295
9                Arizona              4133
10              Michigan              4043
11              New York              4040
12        South Carolina              4022
13              Missouri              3626
14               Alabama              3506
15              Virginia              3422
16               Indiana              3314
17             Louisiana              3181
18              Kentucky              2803
19           Mississippi              2606
20              Oklahoma              2508
21              Colorado        

In [12]:
#Calculate fatalities by state
def groupByStateAndYear(collection_obj):
    pipeline_state = [
        {"$group": {"_id": {"Year":"$YEAR","StateID":"$STATE","State":"$STATENAME"}, "total_fatalities": {"$sum": 1}}},
        {"$sort": {"_id.Year":-1 ,"_id.State": 1}}  # Sort by fatalities descending
    ]
    result = list(collection_obj.aggregate(pipeline_state))
    
    # Convert to DataFrame
    df_state = pd.DataFrame(result)
    
    # Flatten the DataFrame
    df_state = pd.concat([df_state.drop(['_id'], axis=1), df_state['_id'].apply(pd.Series)], axis=1)
    df_state = df_state.reindex(columns=['StateID','Year','State','total_fatalities'])
    print("\nFatalities by State:")
    print(df_state)


In [13]:
groupByStateAndYear(accident)


Fatalities by State:
     StateID  Year          State  total_fatalities
0          1  2022        Alabama               913
1          2  2022         Alaska                75
2          4  2022        Arizona              1183
3          5  2022       Arkansas               583
4          6  2022     California              4109
..       ...   ...            ...               ...
199       51  2019       Virginia               774
200       53  2019     Washington               513
201       54  2019  West Virginia               247
202       55  2019      Wisconsin               527
203       56  2019        Wyoming               120

[204 rows x 4 columns]


In [14]:
# #Calculate fatalities by month
# pipeline_month = [
#     {"$group": {"_id": {"Year":"$YEAR","StateID":"$STATE","State":"$STATENAME", "Month":"$MONTHNAME"}, "total_fatalities": {"$sum": 1}}},
#     {"$sort": {"_id.Year":-1 ,"_id.State":1}}  # Sort by month name alphabetically
# ]
# result = list(accident.aggregate(pipeline_month))
# # Convert to DataFrame
# df_month = pd.DataFrame(result)

# # Flatten the DataFrame
# df_month = pd.concat([df_month.drop(['_id'], axis=1), df_month['_id'].apply(pd.Series)], axis=1)
# df_month = df_month.reindex(columns=['StateID','State','Month','Year','total_fatalities'])
# print("\nFatalities by Month:")
# print(df_month)


In [15]:
#Calculate fatalities by "Monthwise only"
pipeline_month = [
    {"$group": {"_id": {"Year":"$YEAR","MonthID":"$MONTH", "Month":"$MONTHNAME"}, "total_fatalities": {"$sum": 1}}},
    {"$sort": {"_id.MonthID":1,"_id.Year":-1}}  # Sort by month name alphabetically
]
result = list(accident.aggregate(pipeline_month))
# Convert to DataFrame
df_month = pd.DataFrame(result)

# Flatten the DataFrame
df_month = pd.concat([df_month.drop(['_id'], axis=1), df_month['_id'].apply(pd.Series)], axis=1)
df_month = df_month.reindex(columns=['MonthID','Month','Year','total_fatalities'])
print("\nFatalities by Month:")
print(df_month)



Fatalities by Month:
    MonthID      Month  Year  total_fatalities
0         1    January  2022              2946
1         1    January  2021              2837
2         1    January  2020              2486
3         1    January  2019              2476
4         2   February  2022              2748
5         2   February  2021              2351
6         2   February  2020              2453
7         2   February  2019              2205
8         3      March  2022              3050
9         3      March  2021              2916
10        3      March  2020              2375
11        3      March  2019              2544
12        4      April  2022              2951
13        4      April  2021              3285
14        4      April  2020              2130
15        4      April  2019              2611
16        5        May  2022              3376
17        5        May  2021              3440
18        5        May  2020              2876
19        5        May  2019          

In [16]:
#Calculate fatalities by county
pipeline_county = [
    {"$group": {"_id": {"Year":"$YEAR","StateID":"$STATE","State":"$STATENAME","County": "$COUNTYNAME"}, "total_fatalities": {"$sum": 1}}},
    {"$sort": {"_id.State":1,"_id.Year":-1}}  # Sort by county name alphabetically
]
result = list(accident.aggregate(pipeline_county))
# Convert to DataFrame
df_county = pd.DataFrame(result)

# Flatten the DataFrame
df_county = pd.concat([df_county.drop(['_id'], axis=1), df_county['_id'].apply(pd.Series)], axis=1)
df_county  = df_county .reindex(columns=['StateID','State','County','Year','total_fatalities'])
print("\nFatalities by County:")
print(df_county)



Fatalities by County:
       StateID    State            County  Year  total_fatalities
0            1  Alabama     CHEROKEE (19)  2022                11
1            1  Alabama        MACON (87)  2022                 5
2            1  Alabama       AUTAUGA (1)  2022                 9
3            1  Alabama  MONTGOMERY (101)  2022                59
4            1  Alabama    RANDOLPH (111)  2022                 4
...        ...      ...               ...   ...               ...
11456       56  Wyoming      CAMPBELL (5)  2019                 9
11457       56  Wyoming      LARAMIE (21)  2019                11
11458       56  Wyoming     SUBLETTE (35)  2019                 3
11459       56  Wyoming      FREMONT (13)  2019                 7
11460       56  Wyoming      NATRONA (25)  2019                 7

[11461 rows x 5 columns]


In [17]:
#Calculate fatalities by city
pipeline_city = [
    {"$group":{"_id":{"Year":"$YEAR","StateID":"$STATE","State":"$STATENAME","City": "$CITYNAME"}, "total_fatalities": {"$sum": 1}}},
    {"$sort": {"_id.State": 1,"_id.Year": -1}}  # Sort by city name alphabetically
]
result = list(accident.aggregate(pipeline_city))
# Convert to DataFrame
df_city = pd.DataFrame(result)

# Flatten the DataFrame
df_city = pd.concat([df_city.drop(['_id'], axis=1), df_city['_id'].apply(pd.Series)], axis=1)
df_city  = df_city.reindex(columns=['StateID','State','City','Year','total_fatalities'])
print("\nFatalities by City:")
print(df_city)


Fatalities by City:
       StateID    State          City  Year  total_fatalities
0            1  Alabama          ELBA  2022                 1
1            1  Alabama     ALABASTER  2022                 2
2            1  Alabama    LIVINGSTON  2022                 1
3            1  Alabama       RED BAY  2022                 1
4            1  Alabama      BESSEMER  2022                 4
...        ...      ...           ...   ...               ...
21105       56  Wyoming   GREEN RIVER  2019                 1
21106       56  Wyoming      GILLETTE  2019                 3
21107       56  Wyoming       LARAMIE  2019                 1
21108       56  Wyoming  ROCK SPRINGS  2019                 2
21109       56  Wyoming        LANDER  2019                 1

[21110 rows x 5 columns]


In [18]:
# #Calculate fatalities by rural vs urban areas
pipeline_rural_urban = [
    # Add a match stage to filter only Rural and Urban entries
    {"$match": {"RUR_URBNAME": {"$in": ["Rural", "Urban"]}}},
    {"$group":{"_id":{"Year":"$YEAR","StateID":"$STATE","State":"$STATENAME","Rural/Urban":"$RUR_URBNAME"}, "total_fatalities": {"$sum": 1}}},
    {"$sort": {"_id.Year": -1}}
]
result = list(accident.aggregate(pipeline_rural_urban))
# Convert to DataFrame
df_rural_urban = pd.DataFrame(result)

# Flatten the DataFrame
df_rural_urban = pd.concat([df_rural_urban.drop(['_id'], axis=1), df_rural_urban['_id'].apply(pd.Series)], axis=1)
df_rural_urban  = df_rural_urban.reindex(columns=['StateID','State','Year','Rural/Urban','total_fatalities'])
print("\nFatalities by Rural/Urban:")
print(df_rural_urban.head(20))



Fatalities by Rural/Urban:
    StateID        State  Year Rural/Urban  total_fatalities
0        40     Oklahoma  2022       Rural               366
1        47    Tennessee  2022       Rural               492
2        47    Tennessee  2022       Urban               723
3        40     Oklahoma  2022       Urban               277
4        10     Delaware  2022       Rural                73
5        23        Maine  2022       Urban                26
6        12      Florida  2022       Urban              2570
7        10     Delaware  2022       Urban                70
8        39         Ohio  2022       Urban               677
9         8     Colorado  2022       Urban               430
10       34   New Jersey  2022       Rural                62
11       39         Ohio  2022       Rural               496
12       53   Washington  2022       Rural               276
13       23        Maine  2022       Rural               143
14        4      Arizona  2022       Rural               

In [19]:
#Calculate fatalities by Rural/Urban
pipeline_rural_urban = [
    # Add a match stage to filter only Rural and Urban entries
    {"$match": {"RUR_URBNAME": {"$in": ["Rural", "Urban"]}}},
    {"$group":{"_id":{"Year":"$YEAR","Rural/Urban":"$RUR_URBNAME"}, "total_fatalities": {"$sum": 1}}},
    {"$sort": {"_id.Year": -1}}
]
result = list(accident.aggregate(pipeline_rural_urban))
# Convert to DataFrame
df_rural_urban = pd.DataFrame(result)

# Flatten the DataFrame
df_rural_urban = pd.concat([df_rural_urban.drop(['_id'], axis=1), df_rural_urban['_id'].apply(pd.Series)], axis=1)
df_rural_urban  = df_rural_urban.reindex(columns=['Year','Rural/Urban','total_fatalities'])
print("\nFatalities by Rural/Urban:")
print(df_rural_urban)



Fatalities by Rural/Urban:
   Year Rural/Urban  total_fatalities
0  2022       Urban             23482
1  2022       Rural             15541
2  2021       Rural             15548
3  2021       Urban             24099
4  2020       Rural             14734
5  2020       Urban             21047
6  2019       Urban             18753
7  2019       Rural             14626


In [20]:
#Calculate fatalities by time of day (Daytime vs Nighttime)
# pipeline_time_of_day = [
#     {"$group": {"_id":{"Year":"$YEAR","StateID":"$STATE","State":"$STATENAME","Forecast":"$LGT_CONDNAME"}, "total_fatalities": {"$sum": 1}}},
#     {"$sort": {"_id.Year": -1}}
# ]
# result = list(accident.aggregate(pipeline_time_of_day))
# # Convert to DataFrame
# df_time_of_day = pd.DataFrame(result)

# # Flatten the DataFrame
# df_time_of_day = pd.concat([df_time_of_day.drop(['_id'], axis=1), df_time_of_day['_id'].apply(pd.Series)], axis=1)
# df_time_of_day  = df_time_of_day.reindex(columns=['StateID','State','Year','Forecast','total_fatalities'])
# print("\nFatalities by Time of Day:")
# print(df_time_of_day)

In [21]:
#Calculate fatalities by time of day (Daytime vs Nighttime)
pipeline_time_of_day = [
    {"$group": {"_id":{"Year":"$YEAR","Forecast":"$LGT_CONDNAME"}, "total_fatalities": {"$sum": 1}}},
    {"$sort": {"_id.Year": -1}}
]
result = list(accident.aggregate(pipeline_time_of_day))
# Convert to DataFrame
df_time_of_day = pd.DataFrame(result)

# Flatten the DataFrame
df_time_of_day = pd.concat([df_time_of_day.drop(['_id'], axis=1), df_time_of_day['_id'].apply(pd.Series)], axis=1)
df_time_of_day  = df_time_of_day.reindex(columns=['Year','Forecast','total_fatalities'])
print("\nFatalities by Time of Day:")
print(df_time_of_day)


Fatalities by Time of Day:
    Year                 Forecast  total_fatalities
0   2022                    Other                19
1   2022                     Dawn               722
2   2022                     Dusk               910
3   2022      Reported as Unknown               240
4   2022                 Daylight             17674
5   2022       Dark - Not Lighted             10452
6   2022           Dark - Lighted              8524
7   2022             Not Reported                52
8   2022  Dark - Unknown Lighting               628
9   2021           Dark - Lighted              8592
10  2021  Dark - Unknown Lighting               422
11  2021                 Daylight             17986
12  2021       Dark - Not Lighted             10838
13  2021      Reported as Unknown               169
14  2021                     Dusk               940
15  2021             Not Reported               116
16  2021                    Other                26
17  2021                     Dawn   

In [22]:
#Calculate fatalities by Weather
pipeline_weather = [
    {"$group": {"_id":{"Year":"$YEAR","StateID":"$STATE","State":"$STATENAME","Weather":"$WEATHERNAME"}, "total_fatalities": {"$sum": 1}}},
    {"$sort": {"_id.State": 1,"_id.Year":-1}}
]
result = list(accident.aggregate(pipeline_weather))
# Convert to DataFrame
df_weather = pd.DataFrame(result)

# Flatten the DataFrame
df_weather = pd.concat([df_weather.drop(['_id'], axis=1), df_weather['_id'].apply(pd.Series)], axis=1)
df_weather  = df_weather.reindex(columns=['StateID','State','Year','Weather','total_fatalities'])
print("\nFatalities by Weather:")
print(df_weather)


Fatalities by Weather:
      StateID    State  Year           Weather  total_fatalities
0           1  Alabama  2022              Rain                82
1           1  Alabama  2022            Cloudy               146
2           1  Alabama  2022  Fog, Smog, Smoke                11
3           1  Alabama  2022             Clear               670
4           1  Alabama  2022      Blowing Snow                 1
...       ...      ...   ...               ...               ...
1598       56  Wyoming  2019              Rain                 4
1599       56  Wyoming  2019      Blowing Snow                 2
1600       56  Wyoming  2019              Snow                12
1601       56  Wyoming  2019             Clear                93
1602       56  Wyoming  2019  Fog, Smog, Smoke                 1

[1603 rows x 5 columns]


In [23]:
#Calculate fatalities by Co-ordinates
pipeline_coordinates = [
    {"$group": {"_id":{"Year":"$YEAR","StateID":"$STATE","State":"$STATENAME","County": "$COUNTYNAME","City": "$CITYNAME","Latitude":"$LATITUDE","Longitude":"$LONGITUD"}}},
    {"$sort": {"_id.State": 1,"_id.Year":-1}}
]
result = list(accident.aggregate(pipeline_coordinates))
# Convert to DataFrame
df_coordinates = pd.DataFrame(result)

# Flatten the DataFrame
df_coordinates = pd.concat([df_coordinates.drop(['_id'], axis=1), df_coordinates['_id'].apply(pd.Series)], axis=1)
df_coordinates  = df_coordinates.reindex(columns=['StateID','State','Year','County','City','Latitude','Longitude'])
print("\nFatalities by Co-ordinates:")
print(df_coordinates)


Fatalities by Co-ordinates:
        StateID    State  Year         County            City   Latitude  \
0             1  Alabama  2022       LEE (81)          AUBURN  32.584581   
1             1  Alabama  2022    COFFEE (31)      ENTERPRISE  31.291508   
2             1  Alabama  2022       LEE (81)         OPELIKA  32.616242   
3             1  Alabama  2022    GENEVA (61)          GENEVA  31.021992   
4             1  Alabama  2022  PICKENS (107)      CARROLLTON  33.271117   
...         ...      ...   ...            ...             ...        ...   
148225       56  Wyoming  2019     CARBON (7)  NOT APPLICABLE  42.375708   
148226       56  Wyoming  2019     CARBON (7)  NOT APPLICABLE  41.565083   
148227       56  Wyoming  2019     ALBANY (1)  NOT APPLICABLE  41.156164   
148228       56  Wyoming  2019  SHERIDAN (33)  NOT APPLICABLE  44.912142   
148229       56  Wyoming  2019   LARAMIE (21)  NOT APPLICABLE  41.096836   

         Longitude  
0       -85.441367  
1       -85.8485